# Linear Algebra and Optimisation

###### COMP4670/8600 - Statistical Machine Learning - Tutorial

In this lab we will practice minimising a cost function with gradient descent.

### Assumed knowledge
- Linear algebra (see Sam Roweis' notes, linked below, for matrix calculus tips)
- Python programming
- Preferably: Using numpy for matrix calculations (precourse material)

### After this lab, you should be comfortable with:
- Using numpy ndarrays for matrix calculations
- Using scipy.optimise routines to minimise a cost function, with and without a gradient
- Randomly generating input values for testing

## Pre-lab notes
In this lab, you will apply linear algebra to to minimise a cost function in three steps: implementing the cost function, implementing a gradient function, and applying gradient descent. We will be doing this to solve problems throughout the course.

As in all labs, feel free to skip questions if you get stuck, and ask your tutor if you have any questions!

A note on style: in this course we emphasise *functional decomposition* in code style. Avoid using global variables, and remember that often splitting code off into separate functions can make it more readable and testable. (Jupyter notebooks let you call functions defined in previous cells.)

$\newcommand{\trace}[1]{\operatorname{tr}\left\{#1\right\}}$
$\newcommand{\Norm}[1]{\lVert#1\rVert}$
$\newcommand{\RR}{\mathbb{R}}$
$\newcommand{\inner}[2]{\langle #1, #2 \rangle}$
$\newcommand{\DD}{\mathscr{D}}$
$\newcommand{\grad}[1]{\operatorname{grad}#1}$

Setting up python environment (this cell contains Latex macros).

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import scipy.optimize as opt
import time

%matplotlib inline

In an *optimisation problem*, we have a real-valued function $f(X)$ and aim to find an $X$ that minimises/maximises $f$. In machine learning, $f$ is typically a *cost function* or *loss function*, which we want to minimise. This function might measure the error between the indicator values our model predicts and the true values in the training data.

In this lab, we will work with the following toy example. Let's minimise $f: \mathbb{R}^{n \times p} \rightarrow \mathbb{R}$, defined as  
$$
\begin{equation*}
f(X) = \frac{1}{2} \operatorname{tr}({X^T C X N}) + \mu \frac{1}{4} \left\|{N - X^T X}\right\|^2_F,
\end{equation*}
$$
where $C$ is a fixed $n \times n$ symmetric matrix, $\mu$ is a scalar larger than the $p^{th}$ smallest eigenvalue of $C$, and $N$ is a $p \times p$ diagonal matrix with distinct positive entries on the diagonal. We also assume that $n \geq p$.

Let's explain some details of the above you might find unfamiliar.

- $f$ maps an $n \times p$ matrix $X$ to a number on the real line.
- For a square matrix $A$, we write $\operatorname{tr}({A})$ for its trace, which is the sum of all elements on the diagonal of $A$.
- For any matrix $A \in \mathbb{R}^{n \times p}$, define its *Frobenius norm* as $\left\|{A}\right\|_F = \sqrt{\sum_{i=1}^{n} \sum_{j=1}^{p} A_{i,j}^2}$. Can you show that $\left\|{A}\right|_F = \sqrt{\operatorname{tr}({A^T A})} = \sqrt{\operatorname{tr}({A A^T})}$ as well?
- In the definition above, note that only $X$ is the input to $f$; everything else, including $C$, $\mu$ and $N$, are given to us. Therefore, minimising $f$ only involves finding $X$ while keeping everything else fixed.

## Q1. Frobenious Norm

Implement a Python function ```frobenius_norm``` which accepts an arbitrary matrix $A$ and returns
$ \left\|A\right\|_F$. You can choose any formula above.
1. Given a matrix $A \in \mathbb{R}^{n \times p}$, what is the complexity of your implementation of ```frobenius_norm```?
2. Given $n \geq p$, what formula do you think gives the fastest implementation?

### Notes
You can use the following syntax in `numpy`:
- To get the transpose of a matrix `A`, use `A.T`.
- To multiply two matrices `A` and `B`, use `A @ B`.
- To take the sum of all elements of a matrix `A`, use `np.sum(M)` or just `M.sum()`.

In [4]:
# replace this with your solution, add and remove code and markdown cells as appropriate

def frobenius_norm(A):
    return np.sqrt((np.asarray(A) ** 2).sum())

Use the below code to test your ```frobenius_norm``` function:

In [5]:
# Test for frobenius_norm
M = np.array([[0.60094641, 0.9759039 , 0.85339979],
              [0.73835924, 0.34727218, 0.01618439],
              [0.83347728, 0.81740037, 0.36525059],
              [0.62000774, 0.75117202, 0.93941705],
              [0.88817543, 0.37140933, 0.5327329 ]])
print(f'Your frobenius_norm: {frobenius_norm(M):.4f}')
print(f'True frobenius_norm: 2.6918')

Your frobenius_norm: 2.6918
True frobenius_norm: 2.6918


## Q2. Implementing the cost function

Write a Python function, ```cost_function_for_matrix```, which implements the above cost function $f(X)$ defined by
$$
\begin{equation*}
  f(X) = \frac{1}{2} \text{tr}({X^T C X N}) + \mu \frac{1}{4} \left\|{N - X^T X}\right\|^2_F
\end{equation*}
$$
for $X \in \mathbb{R}^{n \times p}$, $n \geq p$.

In [8]:
# replace this with your solution, add and remove code and markdown cells as appropriate
def cost_function_for_matrix(X, C, N, mu):
    return np.trace(X.T @ C @ X @ N)/2 + mu* frobenius_norm(N - X.T @ X) ** 2 /4
    

Use the below code to test your ```cost_function_for_matrix``` function:

In [9]:
# Test for cost_function_for_matrix
X = np.array([[0.09325036, 0.15792007, 0.43645094, 0.95070763, 0.03097754],
              [0.73160312, 0.83306319, 0.02238594, 0.51079686, 0.00742748],
              [0.72548058, 0.80074044, 0.0988811 , 0.28356641, 0.91609969]])
C = np.array([[0.39507301, 0.14470985, 0.29870771],
              [0.14470985, 0.10065113, 0.34081829],
              [0.29870771, 0.34081829, 0.83439717]])
N = np.diag([0.95854111, 0.77966088, 0.18859065, 0.9348394, 0.6822931])
mu = 0.2358
print(f'Your cost_function_for_matrix: {cost_function_for_matrix(X, C, N, mu):.4f}')
print(f'True cost_function_for_matrix: 2.0435')

Your cost_function_for_matrix: 2.0435
True cost_function_for_matrix: 2.0435


## Q3. Cost function with vector argument

The standard optimisation functions we will be using work only for cost functions that take a vector as the varying argument. Write a new function, ```cost_function_for_vector```, that takes $X$ represented as a vector of length $np$ rather than a matrix of dimensions $n\times p$. What arguments will this function take?

In [12]:
# replace this with your solution, add and remove code and markdown cells as appropriate
def cost_function_for_vector(X, C, N, mu, n, p):
    X = X.reshape(n, p)
    return np.trace(X.T @ C @ X @ N)/2 + mu* frobenius_norm(N - X.T @ X) ** 2 /4

Use the below code to test your ```cost_function_for_vector``` function:

In [13]:
# Test for cost_function_for_vector
n = 3
p = 5
# Note that X is a one-dimensional array
X = np.array([0.09325036, 0.15792007, 0.43645094, 0.95070763, 0.03097754, 
              0.73160312, 0.83306319, 0.02238594, 0.51079686, 0.00742748,
              0.72548058, 0.80074044, 0.0988811 , 0.28356641, 0.91609969])
C = np.array([[0.39507301, 0.14470985, 0.29870771],
              [0.14470985, 0.10065113, 0.34081829],
              [0.29870771, 0.34081829, 0.83439717]])
N = np.diag([0.95854111, 0.77966088, 0.18859065, 0.9348394, 0.6822931])
mu = 0.2358
print(f'Your cost_function_for_matrix: {cost_function_for_vector(X, C, N, mu, n, p):.4f}')
print(f'True cost_function_for_matrix: 2.0435')

Your cost_function_for_matrix: 2.0435
True cost_function_for_matrix: 2.0435


## Q4. Minimising the cost function

We will be using two minimisation methods from the ``scipy.optimize`` module.
- First, we will use ``fmin``, which takes an initial guess for $X$ and the function $f$.
- Second, we will use ``fmin_bfgs``, which takes $X$, $f$ and the gradient of $f$. We will write a function to compute the gradient later on.

Since ``fmin_bfgs`` uses additional information bout the function, we should expect to have substantially faster convergence.

### Q4a. Minimising with ```fmin```

Implement a function ```minimise_f_using_fmin``` that, for given values of $C$, $N$ and $\mu$, finds the matrix $X$ that minimizes $f(X)$ using ``fmin``. You will likely need [the docs for ``fmin``](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.fmin.html). Check if your function converges for some (randomly generated) values of $C$, $N$ and $\mu$.

Summary of the docs: if you have a cost function $g(x, y)$ with a fixed value of $y$ and wish to find the value of $x$ that minimises it, the syntax for calling ``fmin`` would be ``fmin(g, x0, args=(y))`` where ``x0`` is an initial guess for the value of $x$, and ``args=(y)`` gives ``fmin`` the rest of the values to pass to the cost function. Note that it is necessary that the variable that can change is the first argument to the cost function.

In [33]:
# replace this with your solution, add and remove code and markdown cells as appropriate
def minimise_f_using_fmin(C, N, mu, n, p, X0):
    """
    parameters:
    initial guess X0
    n by n symmetric matrix C
    p by p diagonal matrix N
    
    returns the X that minimize the cost function f(x)
    """
    return opt.fmin(cost_function_for_vector, X0, args = (C, N, mu, n, p)).reshape(n, p)

In [34]:
# Test for minimise_f_using_fmin
def randomly_generate_values(n, p):
    C = np.random.rand(n,n)
    C = C + C.T
    N = np.diag(np.random.rand(p))
    mu = frobenius_norm(C)
    X0 = np.random.randn(n,p)
    return C, N, mu, X0

n = 3
p = 2
C, N, mu, X0 = randomly_generate_values(n, p)
# print(X0.shape)

minimise_f_using_fmin(C, N, mu, n, p, X0)

Optimization terminated successfully.
         Current function value: -0.016783
         Iterations: 419
         Function evaluations: 648


array([[-0.03789456, -0.5230941 ],
       [ 0.56679162,  0.29412031],
       [-0.61701696,  0.30227598]])

### Q4b. Calculating the gradient of the cost function

To use ``fmin_bfgs``, which is substantially more time efficient, we need to compute the gradient of $f(X)$ with respect to $X$. 
1. Derive a formula for the derivative of $f$ with respect to $X$, $\frac{\partial f}{\partial X}$.
2. Implement a function ``gradient_for_matrix`` that returns the gradient.
3. Implement a function ``gradient_for_vector`` that returns the gradient, then flattens it to an array for use in ``fmin_bfgs``. Remember that this function should take $n$ and $p$ as inputs as well.

You may want to use Sam Roweis' [Matrix Identities](https://cs.nyu.edu/~roweis/notes/matrixid.pdf) and/or the [Matrix Cookbook](https://www.math.uwaterloo.ca/~hwolkowi/matrixcookbook.pdf) as a reference for matrix calculus.

### <span style="color:blue">Answer</span>
<i>--- replace this with your solution, add and remove code and markdown cells as appropriate ---</i>

In [59]:
# replace this with your solution, add and remove code and markdown cells as appropriate
def gradient_for_matrix(X, C, N, mu):
    
    return C @ X @ N - mu * X @ (N - X.T @ X)

def gradient_for_vector(X, C, N, mu, n, p):
    X = X.reshape((n, p))
    
    return gradient_for_matrix(X, C, N, mu).flatten()



### Q4c. Minimising the cost function using the gradient

Write a function ```minimise_f_using_fmin_bfgs``` to minimise $f(X)$ using ```fmin_bfgs```. Have a look at the docs to find the correct syntax. Again, have a try of your function to check that it converges.

In [60]:
# replace this with your solution, add and remove code and markdown cells as appropriate
def minimise_f_using_fmin_bfgs(C, N, mu, n, p, X0):
    return opt.fmin_bfgs(cost_function_for_vector,
                    X0,
                    fprime = gradient_for_vector,
                    args = (C, N, mu, n, p)
                   ).reshape((n, p))

In [61]:
# Test for minimise_f_using_fmin_bfgs
n = 3
p = 2
C, N, mu, X0 = randomly_generate_values(n, p)
minimise_f_using_fmin_bfgs(C, N, mu, n, p, X0)

Optimization terminated successfully.
         Current function value: 0.030391
         Iterations: 20
         Function evaluations: 23
         Gradient evaluations: 23


array([[ 0.0819613 ,  0.29852285],
       [-0.42926028, -0.11583995],
       [ 0.44891588, -0.1652649 ]])

## Q5. Time for convergence

We wish to check whether ``fmin_bfgs`` is actually faster than ``fmin``.
- The most direct way to compare the performance of two algorithms is to compare their wall-clock running time on the same instance.
- Another way is to compare the number of iterations needed to achieve some precision. You can read this off the summary of each algorithm above. For example, for ``fmin_bfgs``, ``Iterations: 295`` means it took ``fmin_bfgs`` 295 iterations until it converged.

We will now compare ``fmin_bfgs`` and ``fmin`` on two instances: one in low-dimensional data and one in high-dimensional data.

### Q5a. Construction of a random matrix $C$ with given eigenvalues

The following function takes in an array ``E`` of $n$ numbers and produces a random $n \times n$ matrix with the same eigenvalues as the numbers in ``E``. It uses a method in linear algebra called [QR decomposition](https://en.wikipedia.org/wiki/QR_decomposition).
1. Given a diagonal matrix $C \in \mathbb{R}^{n \times n}$ with distinct eigenvalues, 
how many different diagonal matrices have the same set of eigenvalues?
2. Can you explain why the following function actually gives you the correct output (a random matrix whose eigenvalues are all in ``E``)?

### <span style="color:blue">Answer</span>
<i>--- replace this with your solution, add and remove code and markdown cells as appropriate ---</i>

In [62]:
def random_matrix_from_eigenvalues(E):
    """Create a square random matrix with a given set of eigenvalues
    E -- list of eigenvalues
    return the random matrix
    """
    n    = len(E)
    # Create a random orthogonal matrix Q via QR decomposition
    # of a random matrix A
    A    = np.random.rand(n,n)
    Q, R = np.linalg.qr(A)
    #  similarity transformation with orthogonal
    #  matrix leaves eigenvalues intact
    #  正交矩阵的相似变换使特征值保持不变
    return Q @ np.diag(E) @ Q.T

### Q5b. Checking convergence time

Is ``fmin_bfgs`` actually faster than ``fmin``? Use the below code to compare runtimes.

Report the runtimes and number of iterations for each algorithm for each instance. Do you see that ``fmin_bfgs`` improves from ``fmin``?

In [63]:
def initialise_low_dimensional_data():
    """Initialise the data, low dimensions"""
    n  = 3
    p  = 2
    mu = 2.7

    N  = np.diag([2.5, 1.5])
    E  = [1, 2, 3]
    C  = random_matrix_from_eigenvalues(E)
    X0 = np.random.rand(n*p)

    return C, N, mu, n, p, X0


def initialise_higher_dimensional_data():
    """Initialise the data, higher dimensions"""
    n  = 20
    p  = 5
    mu = p + 0.5

    N  = np.diag(np.arange(p, 0, -1))
    E  = np.arange(1, n+1)
    C  = random_matrix_from_eigenvalues(E)
    X0 = np.random.rand(n*p)

    return C, N, mu, n, p, X0

def pretty_printing(task_string):
    line_length   = 76
    spaces        = 2
    left_padding  = (line_length - len(task_string)) // 2
    right_padding = line_length - left_padding - len(task_string)
    print("=" * line_length)
    print("=" * (left_padding - spaces) + " " * spaces + task_string + \
            " " * spaces + "=" * (right_padding - spaces))
    print("=" * line_length)    

def run_and_time_all_tests():
    """Run all test and time them using a list of function names"""
    List_of_Test_Names = ["minimise_f_using_fmin",
                 "minimise_f_using_fmin_bfgs"]

    List_of_Initialisations = ["initialise_low_dimensional_data",
                               "initialise_higher_dimensional_data"]

    for test_name in List_of_Test_Names:
        for init_routine in List_of_Initialisations:
            task_string  = test_name + "(" + init_routine + ")"
            pretty_printing(task_string)

            start = time.time()
            C, N, mu, n, p, X0 = globals()[init_routine]()
            exec(test_name+"(C,N,mu,n,p,X0)")
            run_time = time.time() - start
            print("run_time :", run_time)

np.random.seed(42)
run_and_time_all_tests()

=========  minimise_f_using_fmin(initialise_low_dimensional_data)  =========
Optimization terminated successfully.
         Current function value: 3.962963
         Iterations: 528
         Function evaluations: 815
run_time : 0.023898839950561523
=======  minimise_f_using_fmin(initialise_higher_dimensional_data)  ========
run_time : 0.9613921642303467
======  minimise_f_using_fmin_bfgs(initialise_low_dimensional_data)  =======
Optimization terminated successfully.
         Current function value: 3.962963
         Iterations: 14
         Function evaluations: 18
         Gradient evaluations: 18
run_time : 0.0013887882232666016
=====  minimise_f_using_fmin_bfgs(initialise_higher_dimensional_data)  =====
Optimization terminated successfully.
         Current function value: 40.727273
         Iterations: 131
         Function evaluations: 139
         Gradient evaluations: 139
run_time : 0.054717302322387695


## Q6. Minima of $f(X)$

Use the below code to compare the columns $x_1,\dots, x_p$ of the matrix $X^\star$ which minimises $f(X)$ 
$$
\begin{equation*}
  X^\star = \argmin_{X \in \mathbb{R}^{n \times p}} f(X)
\end{equation*}
$$

with the eigenvectors related to the smallest eigenvalues of $C$.

What do you believe this means about $f(X)$?


In [64]:
def normalize_columns(A):
    """Normalise the columns of a 2-D array or matrix to length one
    A - array or matrix (which will be modified)
    """
    if A.ndim != 2:
        raise ValueError("A is not a 2-D array")

    number_of_columns = A.shape[1]
    for i in range(number_of_columns):
        A[:,i] /= np.linalg.norm(A[:,i], ord=2)


def show_results(X_at_min, C):
    """Display the found arg min and compare with eigenvalues of C
    X_at_min -- arguement at minimum found
    C        -- symmetric matrix
    """
    n,p = X_at_min.shape

    normalize_columns(X_at_min)

    # Get the eigenvectors belonging to the smallest eigenvalues
    Eigen_Values, Eigen_Vectors = np.linalg.eig(C)
    Permutation = Eigen_Values.argsort()
    Smallest_Eigenvectors = Eigen_Vectors[:, Permutation[:p]]

    if n < 10:
        print("X_at_min:")
        print(X_at_min)
        print()
        print("Smallest_Eigenvectors:")
        print(Smallest_Eigenvectors)
        print()
    else:
        Project_into_Eigenvectorspace = \
          Smallest_Eigenvectors * Smallest_Eigenvectors.T * X_at_min
        Normal_Component = X_at_min - Project_into_Eigenvectorspace

        print("norm(Normal_Component)/per entry :", \
            np.linalg.norm(Normal_Component, ord=2) / float(n*p))

def show_comparision(num=3):
    for _ in range(num):
        pretty_printing("Comparing X_at_min and Eigenvalues for random values")
        C, N, mu, n, p, X0 = initialise_low_dimensional_data()
        X_at_min = minimise_f_using_fmin_bfgs(C,N,mu,n,p,X0)
        show_results(X_at_min, C)

show_comparision()

==========  Comparing X_at_min and Eigenvalues for random values  ==========
Optimization terminated successfully.
         Current function value: 3.962963
         Iterations: 17
         Function evaluations: 21
         Gradient evaluations: 21
X_at_min:
[[ 0.10208397  0.94578568]
 [ 0.33257976 -0.32456397]
 [ 0.93753377  0.01215205]]

Smallest_Eigenvectors:
[[ 0.10208432  0.94578552]
 [ 0.33257973 -0.3245644 ]
 [ 0.93753374  0.01215281]]

==========  Comparing X_at_min and Eigenvalues for random values  ==========
Optimization terminated successfully.
         Current function value: 3.962963
         Iterations: 14
         Function evaluations: 16
         Gradient evaluations: 16
X_at_min:
[[ 0.51752617 -0.06738513]
 [ 0.73116609 -0.48301698]
 [ 0.44448038  0.87301423]]

Smallest_Eigenvectors:
[[-0.51752588 -0.06738879]
 [-0.73116674 -0.48301241]
 [-0.44447965  0.87301647]]

==========  Comparing X_at_min and Eigenvalues for random values  ==========
Optimization terminated suc